In [1]:
import os
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
current_date = datetime.now().strftime('%Y-%m-%d')
current_date

'2022-02-25'

In [3]:
spark = SparkSession.builder\
     .config('spark.driver.extraClassPath'
            , '/home/user/shared_folder/Distrib/postgresql-42.2.23.jar')\
    .master('local')\
    .appName("homework_7_test")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 18:37:57 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/25 18:37:57 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/25 18:37:57 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
bronze_aisles_df = spark.read\
    .load(os.path.join('/', 'datalake', 'bronze', 'dshop', 'aisles', current_date, 'aisles.csv'),
          header = "true",
          inferSchema = "true",
          format = "csv"
         )

In [6]:
bronze_aisles_df.count()

134

In [7]:
bronze_aisles_df.printSchema()

root
 |-- aisle_id: integer (nullable = true)
 |-- aisle: string (nullable = true)



In [8]:
bronze_aisles_df = bronze_aisles_df\
    .where(F.col('aisle').isNotNull())

In [9]:
bronze_aisles_df.show()

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
|       3| energy granola bars|
|       4|       instant foods|
|       5|marinades meat pr...|
|       6|               other|
|       7|       packaged meat|
|       8|     bakery desserts|
|       9|         pasta sauce|
|      10|    kitchen supplies|
|      11|    cold flu allergy|
|      12|         fresh pasta|
|      13|      prepared meals|
|      14|tofu meat alterna...|
|      15|    packaged seafood|
|      16|         fresh herbs|
|      17|  baking ingredients|
|      18|bulk dried fruits...|
|      19|       oils vinegars|
|      20|        oral hygiene|
+--------+--------------------+
only showing top 20 rows



In [10]:
bronze_aisles_df = bronze_aisles_df.dropDuplicates()

In [11]:
bronze_aisles_df.count()

134

In [12]:
bronze_aisles_df.write\
    .parquet(os.path.join('/', 'datalake', 'silver', 'dshop', 'aisles'), mode = 'overwrite')